In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split

###  Chargement des données
---

In [2]:
df = pd.read_csv('data/get_around_pricing_project.csv', encoding='utf-8')
df.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


### L'EDA Analyse exploratoire des données Basique
---

-Unnamed: 0 → Index<br>
-model_key → Clé du modèle<br>
-mileage → Kilométrage<br>
-engine_power → Puissance du moteur<br>
-fuel → Carburant<br>
-paint_color → Couleur de la peinture<br>
-car_type → Type de voiture<br>
-private_parking_available → Parking privé disponible<br>
-has_gps → A GPS<br>
-has_air_conditioning → A climatisation<br>
-automatic_car → Voiture automatique<br>
-has_getaround_connect → A Getaround Connect<br>
-has_speed_regulator → A régulateur de vitesse<br>
-winter_tires → Pneus hiver<br>
-rental_price_per_day → Prix de location par jour<br>

In [3]:
print(f"Nous avons {df.shape[1]} colonnes et {df.shape[0]} lignes dans ce jeu de données")

print("\n... Informations sur les colonnes et les types de données ...")
df.info()

print("\n... Statistiques descriptives sur les colonnes numériques ...")
display(df.describe())

print("\n... Valeurs manquantes ...")
nb_valeurs_manquantes = df.isnull().sum()
pourcentage_valeurs_manquantes = (df.isnull().mean() * 100).round(2)
df_manquants = pd.DataFrame({
    'Nombre de Valeurs Manquantes': nb_valeurs_manquantes,
    '% Valeurs Manquantes': pourcentage_valeurs_manquantes
})
print(df_manquants[df_manquants['Nombre de Valeurs Manquantes'] > 0]
      .sort_values('Nombre de Valeurs Manquantes', ascending=False))




Nous avons 15 colonnes et 4843 lignes dans ce jeu de données

... Informations sur les colonnes et les types de données ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 4843 non-null   int64 
 1   model_key                  4843 non-null   object
 2   mileage                    4843 non-null   int64 
 3   engine_power               4843 non-null   int64 
 4   fuel                       4843 non-null   object
 5   paint_color                4843 non-null   object
 6   car_type                   4843 non-null   object
 7   private_parking_available  4843 non-null   bool  
 8   has_gps                    4843 non-null   bool  
 9   has_air_conditioning       4843 non-null   bool  
 10  automatic_car              4843 non-null   bool  
 11  has_getaround_connect      4843 non-null   bool  

,Unnamed: 0,mileage,engine_power,rental_price_per_day
count,4843.000000,4.843000e+03,4843.00000,4843.000000
mean,2421.000000,1.409628e+05,128.98823,121.214536
std,1398.198007,6.019674e+04,38.99336,33.568268
min,0.000000,-6.400000e+01,0.00000,10.000000
25%,1210.500000,1.029135e+05,100.00000,104.000000
50%,2421.000000,1.410800e+05,120.00000,119.000000
75%,3631.500000,1.751955e+05,135.00000,136.000000
max,4842.000000,1.000376e+06,423.00000,422.000000



... Valeurs manquantes ...
Empty DataFrame
Columns: [Nombre de Valeurs Manquantes, % Valeurs Manquantes]
Index: []


### L'EDA Analyse exploratoire des données Approfondie
---

In [4]:
# Marques de véhicules les plus loués
print("\n... Marques de véhicules les plus loués ...")
print("Les 3 marques les plus louées sont : Citroên, Renault et BMW.")
df_model_counts = df['model_key'].value_counts().reset_index()
df_model_counts.columns = ['model_key', 'counts']
print(df_model_counts)

fig = px.bar(df_model_counts.head(10),
             x='model_key', y='counts',
             title='Top 10 marques de véhicules les plus loués')
fig.show()

# Type de carburant
print("\n... Type de carburant ...")
print("Le carburant le plus loué est le Diesel.")
df_fuel_counts = df['fuel'].value_counts().reset_index()
df_fuel_counts.columns = ['fuel', 'counts']
print(df_fuel_counts)

fig = px.bar(df_fuel_counts,
             x='fuel', y='counts',
             title='Répartition des types de carburant')
fig.show()

# Puissance moteur
print("\n... Analyse de la puissance moteur ...")
mean_power = df['engine_power'].mean()
std_power = df['engine_power'].std()
print(f"Les locations de voitures sont louées avec une puissance moyenne de {mean_power:.1f}ch\npour une écart type {std_power:.1f} ch")

fig = px.histogram(df,
                   x='engine_power',
                   nbins=10,
                   title='Distribution des puissances moteur')
fig.add_vline(x=mean_power, line_color="red", annotation_text="Moyenne")
fig.show()

# Prix de location 
print("\n... Analyse des prix de location par jour ...")
median_price = df['rental_price_per_day'].median()
q1_price = df['rental_price_per_day'].quantile(0.25)
q3_price = df['rental_price_per_day'].quantile(0.75)
print(f"Le prix médian d'une loctaion est de {median_price:.2f} €")

# Visualisation prix loc 
fig = px.histogram(df,
                   x='rental_price_per_day',
                   nbins=20,
                   title='Distribution du prix de location par jour',
                   labels={"rental_price_per_day": "Prix (€)"})

fig.add_vline(x=median_price,
              line=dict(color="red", dash="dash"),
              annotation_text="Médiane",
              annotation_position="top")

fig.show()

# Prix médian par marque 
print("\n... Prix médian par marque ...")
print("La marque Suzuki présente le prix médian de location le plus élevé à 214 €,\nmais ce chiffre n’est pas représentatif, car il ne repose que sur 8 locations dans le jeu de données.")
median_prices = (df
                 .groupby('model_key')['rental_price_per_day']
                 .median()
                 .sort_values(ascending=False)
                 .reset_index())

# Visualisation
import plotly.express as px

fig = px.bar(median_prices,
             x='model_key',
             y='rental_price_per_day',
             title='Prix médian de location - Toutes les marques',
             labels={'rental_price_per_day': 'Prix médian (€)', 'model_key': 'Marque'},
             color='rental_price_per_day',
             color_continuous_scale='Greens')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

# Prix par marque TOP 10
print("\n... Prix médian par marque (Top 10) ...")
print("Parmi les 10 marques les plus représentées, Mitsubishi affiche le prix médian de location le plus élevé à 184 €,\nsuivi de Volkswagen avec une médiane de 141 €.")
top_10 = df_model_counts['model_key'].head(10)
df_top_10 = df[df['model_key'].isin(top_10)]

median_prices = (df_top_10
                 .groupby('model_key')['rental_price_per_day']
                 .median()
                 .sort_values(ascending=False)
                 .reset_index())

fig = px.bar(median_prices,
             x='model_key',
             y='rental_price_per_day',
             title='Prix médian de location - Top 10 marques',
             labels={'rental_price_per_day': 'Prix médian (€)', 'model_key': 'Marque'},
             color='rental_price_per_day',
             color_continuous_scale='Greens')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

print("La variabilité des prix diffère selon les marques : certaines, comme BMW, Toyota ou Renault,\nprésentent une large dispersion avec des tarifs très bas et très élevés,\ntandis que  Mercedes et Volkswagen, affichent des prix plus homogènes.")
fig = px.violin(df_top_10,
                x="model_key", y="rental_price_per_day",
                box=True, points="suspectedoutliers",
                color="model_key",
                title="Distribution des prix par marque (Top 10)",
                labels={'rental_price_per_day': 'Prix par jour (€)', 'model_key': 'Marque'})
fig.update_layout(showlegend=False, xaxis_tickangle=-45)
fig.show()

# car_type
print("\n... Modéle Véhicule ...")
print("Break, Berline et SUV dominent le marché : avec respectivement 1606, 1168 et 1058 unités,\nces trois types de véhicules représentent 80% des locations sur ce jeu de données")
model=df['car_type'].value_counts()
print(model)

# Dictionnaire de traduction
translation = {
    'estate': 'Break',
    'sedan': 'Berline',
    'suv': 'SUV',
    'hatchback': 'Compacte',
    'subcompact': 'Sous-compacte',
    'coupe': 'Coupé',
    'convertible': 'Cabriolet',
    'van': 'Van'
}

model_counts = df['car_type'].value_counts().reset_index()
model_counts.columns = ['car_type', 'count']

# Ajouter la colonne traduction
model_counts['car_type_fr'] = model_counts['car_type'].map(translation)

# Visualisation
fig = px.bar(model_counts, 
             x='car_type_fr',  
             y='count',
             title='Distribution des modèles de véhicules')
fig.show()

# Comptage par modèle et type de voiture
print("Les 4 marques les plus présentes sur le marché des Break, Berline et SUV sont : Audi, Citroën, Renault et Peugeot. ")

model_type_counts = df.groupby(['model_key', 'car_type']).size().reset_index(name='count')

# Ajouter les noms français
model_type_counts['car_type_fr'] = model_type_counts['car_type'].map(translation)

# Histogramme empilé
fig = px.bar(model_type_counts,
             x='model_key',
             y='count',
             color='car_type_fr',
             title='Répartition des types de véhicules par modèle',
             text='count')

fig.update_layout(barmode='stack', xaxis_title='Modèle', yaxis_title='Nombre de véhicules')
fig.show()


... Marques de véhicules les plus loués ...
Les 3 marques les plus louées sont : Citroên, Renault et BMW.
      model_key  counts
0       Citroën     969
1       Renault     916
2           BMW     827
3       Peugeot     642
4          Audi     526
5        Nissan     275
6    Mitsubishi     231
7      Mercedes      97
8    Volkswagen      65
9        Toyota      53
10         SEAT      46
11       Subaru      44
12         Opel      33
13      Ferrari      33
14          PGO      33
15     Maserati      18
16       Suzuki       8
17      Porsche       6
18         Ford       5
19   KIA Motors       3
20   Alfa Romeo       3
21         Fiat       2
22        Lexus       2
23  Lamborghini       2
24         Mini       1
25        Mazda       1
26        Honda       1
27       Yamaha       1



... Type de carburant ...
Le carburant le plus loué est le Diesel.
            fuel  counts
0         diesel    4641
1         petrol     191
2  hybrid_petrol       8
3        electro       3



... Analyse de la puissance moteur ...
Les locations de voitures sont louées avec une puissance moyenne de 129.0ch
pour une écart type 39.0 ch



... Analyse des prix de location par jour ...
Le prix médian d'une loctaion est de 119.00 €



... Prix médian par marque ...
La marque Suzuki présente le prix médian de location le plus élevé à 214 €,
mais ce chiffre n’est pas représentatif, car il ne repose que sur 8 locations dans le jeu de données.



... Prix médian par marque (Top 10) ...
Parmi les 10 marques les plus représentées, Mitsubishi affiche le prix médian de location le plus élevé à 184 €,
suivi de Volkswagen avec une médiane de 141 €.


La variabilité des prix diffère selon les marques : certaines, comme BMW, Toyota ou Renault,
présentent une large dispersion avec des tarifs très bas et très élevés,
tandis que  Mercedes et Volkswagen, affichent des prix plus homogènes.



... Modéle Véhicule ...
Break, Berline et SUV dominent le marché : avec respectivement 1606, 1168 et 1058 unités,
ces trois types de véhicules représentent 80% des locations sur ce jeu de données
car_type
estate         1606
sedan          1168
suv            1058
hatchback       699
subcompact      117
coupe           104
convertible      47
van              44
Name: count, dtype: int64


Les 4 marques les plus présentes sur le marché des Break, Berline et SUV sont : Audi, Citroën, Renault et Peugeot. 
